In [7]:
# 导入必要的模块
import itertools
import json
from typing import List
from tqdm import tqdm

# 假设这些模块已经在项目中正确导入
from src.HandPoseRecorder import HandPoseRecordPool, HandPoseRecorder, RightHandRecorder
from src.animate.animate import leftHand2Animation, rightHand2Animation, ElectronicRightHand2Animation, addPitchwheel, animated_guitar_string
from src.guitar.Guitar import Guitar
from src.guitar.GuitarString import createGuitarStrings
from src.guitar.MusicNote import MusicNote
from src.hand.LeftFinger import LeftFinger
from src.hand.LeftHand import LeftHand
from src.hand.RightHand import RightHand, generatePossibleRightHands
from src.midi.midiToNote import calculate_frame, get_tempo_changes, midiToGuitarNotes, processedNotes
from src.utils.utils import convertChordTofingerPositions, convertNotesToChord

# 定义参数（可以根据需要修改）
avatar = 'Furina_E'
midi_name = 'エケステンドアッシュ-蓬莱人'
midiFilePath = f"asset/midi/{midi_name}.mid"
track_number = [3]  # 可以根据需要修改
channel_number = -1  
FPS = 30
guitar_string_notes = ["d", "b", "G", "D", "A", "D1"]
octave_down_checkbox = False
capo_number = 0

# 第一步：初始化文件路径和基本参数
filename = midiFilePath.split("/")[-1].split(".")[0]
track_number_string = "_".join([str(i) for i in track_number])
notes_map_file = f"output/{filename}_{track_number_string}_notes_map.json"
messages_file = f"output/{filename}_{track_number_string}_messages.json"
left_hand_recorder_file = f"output/{filename}_{track_number_string}_lefthand_recorder.json"
left_hand_animation_file = f"output/{avatar}_{filename}_{track_number_string}_lefthand_animation.json"
right_hand_recorder_file = f"output/{filename}_{track_number_string}_righthand_recorder.json"
right_hand_animation_file = f"output/{avatar}_{filename}_{track_number_string}_righthand_animation.json"
guitar_string_recorder_file = f"output/{filename}_{track_number_string}_guitar_string_recorder.json"

print("文件路径已初始化:")
print(f"  音符映射文件: {notes_map_file}")
print(f"  消息文件: {messages_file}")
print(f"  左手记录文件: {left_hand_recorder_file}")
print(f"  左手动画文件: {left_hand_animation_file}")
print(f"  右手记录文件: {right_hand_recorder_file}")
print(f"  右手动画文件: {right_hand_animation_file}")
print(f"  吉他弦动画文件: {guitar_string_recorder_file}")

# 第二步：解析MIDI文件获取音符和速度变化信息
tempo_changes, ticks_per_beat = get_tempo_changes(midiFilePath)
notes_map, pitch_wheel_map, messages = midiToGuitarNotes(
    midiFilePath, useTracks=track_number, useChannel=channel_number,
    octave_down_checkbox=octave_down_checkbox, capo_number=capo_number)

# 保存notes_map和messages到文件
with open(notes_map_file, "w") as f:
    json.dump(notes_map, f, indent=4)
with open(messages_file, "w") as f:
    json.dump(messages, f, indent=4)

print(f'全曲的速度变化是:')
for track, tempo, tick in tempo_changes:
    print(f'在{track}轨，tick为{tick}时，速度变为{tempo}')

print(f'\n全曲的每拍tick数是:{ticks_per_beat}\n')

# 计算总时长
total_tick = notes_map[-1]['real_tick']
total_frame = calculate_frame(tempo_changes, ticks_per_beat, FPS, total_tick)
total_time = total_frame/FPS
print(f'如果以{FPS}的fps做成动画，一共是{total_tick} ticks, 合计{total_frame}帧, 约{total_time}秒')

# 第三步：初始化吉他和左手
guitar_string_list = createGuitarStrings(guitar_string_notes)
max_string_index = len(guitar_string_list) - 1

# 初始化吉他
guitar = Guitar(guitar_string_list)

# 设定各手指初始状态
leftFingers = [
    LeftFinger(1, guitar_string_list[2], 1),
    LeftFinger(2, guitar_string_list[2], 2),
    LeftFinger(3, guitar_string_list[2], 3),
    LeftFinger(4, guitar_string_list[2], 4)
]

# 初始化左手
initLeftHand = LeftHand(leftFingers)

# 初始化第一个记录器
handPoseRecord = HandPoseRecorder()
handPoseRecord.addHandPose(initLeftHand, 0, 0)

# 初始化记录池
handPoseRecordPool = HandPoseRecordPool(100)
handPoseRecordPool.insert_new_hand_pose_recorder(handPoseRecord, 0)

print("吉他和左手初始化完成")
print(f"最大弦索引: {max_string_index}")
print(
    f"初始手指位置: {[f'手指{f._fingerIndex}在{f.stringIndex}弦{f.fret}品' for f in leftFingers]}")


def update_recorder_pool(total_steps: int, guitar: Guitar, handPoseRecordPool: HandPoseRecordPool, notes_map, current_recoreder_num, previous_recoreder_num):
    with tqdm(total=total_steps, desc="Processing", ncols=100, unit="step") as progress:
        for i in range(0, total_steps):
            guitarNote = notes_map[i]
            current_recoreder_num, previous_recoreder_num = generateLeftHandRecoder(
                guitarNote, guitar, handPoseRecordPool, current_recoreder_num, previous_recoreder_num)
            progress.update(1)


def generateLeftHandRecoder(guitarNote, guitar: Guitar, handPoseRecordPool: HandPoseRecordPool, current_recoreder_num: int, previous_recoreder_num: int):
    notes = guitarNote.get("notes", False)
    if notes == False:
        return current_recoreder_num, previous_recoreder_num
    real_tick = guitarNote["real_tick"]
    min_note = guitar.guitarStrings[-1].getBaseNote()
    max_note = guitar.guitarStrings[0].getBaseNote() + 22
    notes = processedNotes(notes, min_note, max_note)

    # calculate all possible chords, including the position information of notes on the guitar. 计算所有可能的和弦,包含音符在吉它上的位置信息。
    chords = convertNotesToChord(notes, guitar)

    # init current record list. 记录池先更新初始化当前记录列表。
    handPoseRecordPool.readyForRecord()
    fingerPositionsList = []
    handPoseRecordCount = 0

    # calculate all possible fingerings, including the position information of fingers on the guitar. 计算所有可能的按法，包含手指在吉它上的位置信息。
    for chord in chords:
        possibleFingerPositions = convertChordTofingerPositions(chord)
        if len(possibleFingerPositions) == 0:
            continue
        fingerPositionsList += possibleFingerPositions

    if len(fingerPositionsList) == 0:
        print(
            f"当前时间是{real_tick}，当前notes是{notes},没有找到合适的按法。这是所有的chords：{chords}。")

    for handPoseRecord, fingerPositions in itertools.product(handPoseRecordPool.preHandPoseRecordPool, fingerPositionsList):
        oldhand = handPoseRecord.currentHandPose()

        # Iterate through the list of fingerings, generate a new LeftHand object based on the fingering. 遍历按法列表，根据按法生成新的LeftHand对象。
        all_fingers, entropy, use_barre = oldhand.generateNextHands(
            guitar, fingerPositions)

        if all_fingers is not None:
            new_entropy = handPoseRecord.currentEntropy + entropy
            insert_index = handPoseRecordPool.check_insert_index(new_entropy)
            # 当新手型符合插入记录器条件时
            if insert_index != -1:
                newHandPoseRecord = HandPoseRecorder()
                new_hand = LeftHand(all_fingers, use_barre)

                newHandPoseRecord.handPoseList = handPoseRecord.handPoseList + \
                    [new_hand]
                newHandPoseRecord.currentEntropy = handPoseRecord.currentEntropy + entropy
                newHandPoseRecord.entropys = handPoseRecord.entropys + \
                    [new_entropy]
                newHandPoseRecord.real_ticks = handPoseRecord.real_ticks + \
                    [real_tick]

                handPoseRecordPool.insert_new_hand_pose_recorder(
                    newHandPoseRecord, insert_index)
                handPoseRecordCount += 1

    previous_recoreder_num = current_recoreder_num
    current_recoreder_num = len(handPoseRecordPool.curHandPoseRecordPool)

    if current_recoreder_num < previous_recoreder_num:
        print(
            f"当前record数量是{current_recoreder_num}，上一次record数量是{previous_recoreder_num}，这一轮操作一共append了{handPoseRecordCount}个record。")
        print(f"此时的real_tick是{real_tick},此时的notes是：{notes},对应的音符是：")
        for note in notes:
            musice_note = MusicNote(note)
            print(musice_note.key)

    # 如果无法生成正常的按法，就添加一个默认在第五品的A和弦按法。
    # sunflower中第五把位Am按法 + 右手食指12品击弦，产生的音符是常规按法所无法生成的，所以这里用默认按法来填充。
    if current_recoreder_num == 0:
        defalut_fingers = [
            LeftFinger(1, guitar.guitarStrings[5], 5, 'Barre'),
            LeftFinger(1, guitar.guitarStrings[0], 5, 'Barre'),
            LeftFinger(1, guitar.guitarStrings[1], 5, 'Barre'),
            LeftFinger(2, guitar.guitarStrings[2], 6, 'Pressed'),
            LeftFinger(3, guitar.guitarStrings[4], 7, 'Pressed'),
            LeftFinger(4, guitar.guitarStrings[3], 7, 'Pressed')
        ]
        defalut_hand = LeftHand(defalut_fingers)
        for handrecoder in handPoseRecordPool.preHandPoseRecordPool:
            new_handrecoder = HandPoseRecorder()
            new_handrecoder.handPoseList = handrecoder.handPoseList + \
                [defalut_hand]
            new_handrecoder.currentEntropy = handrecoder.currentEntropy
            new_handrecoder.entropys = handrecoder.entropys + [0]
            new_handrecoder.real_ticks = handrecoder.real_ticks + [real_tick]
            handPoseRecordPool.insert_new_hand_pose_recorder(
                new_handrecoder, 0)

    return current_recoreder_num, previous_recoreder_num


def leftHand2ElectronicRightHand(left_hand_recorder_file, right_hand_recorder_file):
    result = []
    with open(left_hand_recorder_file, "r") as f:
        data = json.load(f)
        total_steps = len(data)

        with tqdm(total=total_steps, desc="Processing", ncols=100, unit="step") as progress:
            for i in range(total_steps):
                item = data[i]
                pitchwheel = item.get("pitchwheel", 0)
                if pitchwheel != 0:
                    continue
                leftHand = item["leftHand"]
                frame = item["frame"]
                strings = []
                for finger in leftHand:
                    if finger["fingerIndex"] == -1 or 0 < finger["fingerInfo"]["press"] < 5:
                        strings.append(finger["fingerInfo"]['stringIndex'])

                if len(strings) > len(set(strings)):
                    strings = list(set(strings))

                result.append({
                    'frame': frame,
                    'strings': strings,
                })

                progress.update(1)

    with open(right_hand_recorder_file, "w") as f:
        json.dump(result, f, indent=4)


def generateRightHandRecoder(item, rightHandRecordPool, current_recoreder_num, previous_recoreder_num, max_string_index):
    real_tick = item["real_tick"]
    leftHand = item["leftHand"]
    touchedStrings = []
    lower_strings = []

    for finger in leftHand:
        if finger["fingerIndex"] == -1 or 5 > finger["fingerInfo"]["press"] > 0:
            string_index = finger["fingerInfo"]['stringIndex']
            touchedStrings.append(string_index)
            if string_index > 2:
                lower_strings.append(string_index)


def update_right_hand_recorder_pool(left_hand_recorder_file, rightHandRecordPool, current_recoreder_num, previous_recoreder_num, max_string_index):
    with open(left_hand_recorder_file, "r") as f:
        data = json.load(f)
        total_steps = len(data)
        current_recoreder_num = 0
        previous_recoreder_num = current_recoreder_num

        with tqdm(total=total_steps, desc="Processing", ncols=100, unit="step") as progress:
            for i in range(total_steps):
                item = data[i]
                generateRightHandRecoder(
                    item, rightHandRecordPool, current_recoreder_num, previous_recoreder_num, max_string_index)
                progress.update(1)

文件路径已初始化:
  音符映射文件: output/エケステンドアッシュ-蓬莱人_3_notes_map.json
  消息文件: output/エケステンドアッシュ-蓬莱人_3_messages.json
  左手记录文件: output/エケステンドアッシュ-蓬莱人_3_lefthand_recorder.json
  左手动画文件: output/Furina_E_エケステンドアッシュ-蓬莱人_3_lefthand_animation.json
  右手记录文件: output/エケステンドアッシュ-蓬莱人_3_righthand_recorder.json
  右手动画文件: output/Furina_E_エケステンドアッシュ-蓬莱人_3_righthand_animation.json
  吉他弦动画文件: output/エケステンドアッシュ-蓬莱人_3_guitar_string_recorder.json
全曲的速度变化是:
在0轨，tick为0时，速度变为384615

全曲的每拍tick数是:480

如果以30的fps做成动画，一共是245280 ticks, 合计5896.14795帧, 约196.538265秒
吉他和左手初始化完成
最大弦索引: 5
初始手指位置: ['手指1在2弦1品', '手指2在2弦2品', '手指3在2弦3品', '手指4在2弦4品']


In [2]:
# 第四步：生成左手按弦数据
current_recoreder_num = 0
previous_recoreder_num = 0
total_steps = len(notes_map)

print('开始生成左手按弦数据')
update_recorder_pool(total_steps, guitar, handPoseRecordPool, notes_map, current_recoreder_num, previous_recoreder_num)

# 获取最优解
bestHandPoseRecord = handPoseRecordPool.curHandPoseRecordPool[0]
bestEntropy = bestHandPoseRecord.currentEntropy
print(f"最小消耗熵为：{bestEntropy}")
bestHandPoseRecord.save(left_hand_recorder_file, tempo_changes, ticks_per_beat, FPS)
print(f"总音符数应该为{total_steps}")
print(f"实际输出音符数为{len(bestHandPoseRecord.handPoseList)}")

开始生成左手按弦数据


Processing:  12%|█████▌                                        | 137/1122 [00:04<00:34, 28.87step/s]


KeyboardInterrupt: 

In [3]:
# 第五步：处理推弦动作（如果有）
if len(pitch_wheel_map) > 0:
    print("检测到推弦动作，正在处理...")
    for item in pitch_wheel_map:
        frame = calculate_frame(tempo_changes, ticks_per_beat, FPS, item['real_tick'])
        item['frame'] = frame
    addPitchwheel(left_hand_recorder_file, pitch_wheel_map)
    print(f"已处理{len(pitch_wheel_map)}个推弦动作")
else:
    print("未检测到推弦动作")

未检测到推弦动作


In [8]:
# 第六步：生成左手动画数据
print("正在生成左手动画数据...")
leftHand2Animation(avatar, left_hand_recorder_file,
                   left_hand_animation_file, tempo_changes, ticks_per_beat, FPS, max_string_index)
print("左手动画数据生成完成")

正在生成左手动画数据...
左手动画数据生成完成


In [3]:
# 第七步：生成右手演奏数据
print('开始生成右手演奏数据')
if avatar.endswith("_E"):
    print("检测到电吉他角色，使用电吉他右手处理...")
    leftHand2ElectronicRightHand(
        left_hand_recorder_file, right_hand_recorder_file)
    ElectronicRightHand2Animation(
        avatar, right_hand_recorder_file, right_hand_animation_file, FPS)
    print("电吉他右手动画数据生成完成")
else:
    print("使用古典吉他右手处理...")
    initRightHand = RightHand(
        usedFingers=[], rightFingerPositions=[max_string_index, 2, 1, 0], preUsedFingers=[])

    initRightHandRecorder = RightHandRecorder()
    initRightHandRecorder.addHandPose(initRightHand, 0, 0)

    rightHandRecordPool = HandPoseRecordPool(100)
    rightHandRecordPool.insert_new_hand_pose_recorder(
        initRightHandRecorder, 0)

    current_recoreder_num = 0
    previous_recoreder_num = current_recoreder_num

    update_right_hand_recorder_pool(
        left_hand_recorder_file, rightHandRecordPool, current_recoreder_num, previous_recoreder_num, max_string_index)

    # 获取最优解
    bestHandPoseRecord = rightHandRecordPool.curHandPoseRecordPool[0]
    bestEntropy = bestHandPoseRecord.currentEntropy
    print(f"最小消耗熵为：{bestEntropy}")
    bestHandPoseRecord.save(right_hand_recorder_file,
                            tempo_changes, ticks_per_beat, FPS)

    rightHand2Animation(avatar, right_hand_recorder_file,
                        right_hand_animation_file, FPS)
    print("古典吉他右手动画数据生成完成")

开始生成右手演奏数据
检测到电吉他角色，使用电吉他右手处理...


Processing: 100%|████████████████████████████████████████| 1122/1122 [00:00<00:00, 1122348.94step/s]


电吉他右手动画数据生成完成


In [4]:
# 第八步：生成吉他弦动画数据
print('开始生成吉他弦动画数据...')
animated_guitar_string(left_hand_recorder_file,
                       guitar_string_recorder_file, FPS)
print("吉他弦动画数据生成完成")

开始生成吉他弦动画数据...
吉他弦动画数据生成完成


In [5]:
# 最终结果输出
finall_info = f'''全部执行完毕:
recorder文件被保存到了:
  - {left_hand_recorder_file}
  - {right_hand_recorder_file}
动画文件被保存到了:
  - {left_hand_animation_file}
  - {right_hand_animation_file}
吉它弦动画文件被保存到了:
  - {guitar_string_recorder_file}'''

print(finall_info)

全部执行完毕:
recorder文件被保存到了:
  - output/エケステンドアッシュ-蓬莱人_3_lefthand_recorder.json
  - output/エケステンドアッシュ-蓬莱人_3_righthand_recorder.json
动画文件被保存到了:
  - output/Furina_E_エケステンドアッシュ-蓬莱人_3_lefthand_animation.json
  - output/Furina_E_エケステンドアッシュ-蓬莱人_3_righthand_animation.json
吉它弦动画文件被保存到了:
  - output/エケステンドアッシュ-蓬莱人_3_guitar_string_recorder.json
